# **0. Imports**

In [1]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.applications.resnet50 import ResNet50
import pandas as pandas
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

# **1. Hyperparameters**

In [2]:
img_height = 200
img_width = 267
img_channels = 3

learning_rate = 1e-7
batch_size = 5
database_len_train = 580
database_len_val = 100

nb_epoch = 20
# Number of batches
steps_per_epoch = np.ceil( database_len_train / batch_size )
validation_steps = np.ceil( database_len_val / batch_size )

# **2. Dataset preparation**

In [3]:
#route = "/content/drive/MyDrive/TFM/data" 
route = "/content/drive/MyDrive/TFM/data_reduced" 

# This function returns the generator of the form: (inputs, targets)               
def load_data(Train_df, idx, batch_size):
        
    parking_import = pandas.read_csv( Train_df + "/groundtruth.txt", skiprows=idx*batch_size, nrows=batch_size, delim_whitespace=True)
    parking_data = np.array(parking_import.values)

    batch_images = []
    batch_labels = []  

    for i in range (len(parking_data)):    
        image_name = parking_data[i][0]            
        image_name =  Train_df + "/images/" + image_name        
        img = load_img(image_name, color_mode="rgb", target_size=(img_height, img_width), interpolation="nearest")     
        img = img_to_array(img)        
        batch_images.append(img)         
        batch_labels.append(parking_data[i][1:])
    
    batch_images = np.array(batch_images)
    batch_images = batch_images / 255.0
    batch_labels = np.array(batch_labels).astype('float32')
    
    return (batch_images, batch_labels)           


def batch_generator(Train_df, batch_size, steps):    
    idx = 1
    while True:
        
        yield load_data(Train_df, idx-1, batch_size)## Yields data
        
        if idx < steps:
            idx+=1
        else:
            idx=1
            
### Generator objects for train and validation
my_training_batch_generator = batch_generator( route + "/train", batch_size, steps_per_epoch)
my_validation_batch_generator = batch_generator(route + "/val", batch_size, validation_steps)


   

# **3. Model definition**

In [4]:
##################### RESNET 50 MODEL #########################################
img_input = Input(shape=(img_height, img_width, img_channels)) 


model = ResNet50( include_top=False, weights='imagenet',  input_tensor = img_input) 
#model = keras.applications.Xception( include_top=False, weights='imagenet', input_tensor=img_input)
#model = keras.applications.NASNetLarge( include_top=False, weights='imagenet', input_tensor=img_input)

x = model.output

# FC layers
x = Flatten()(x)
x = Dense(1024)(x)
x = Activation('sigmoid')(x)
x = Dropout(0.5)(x)

# Output dimension (empty place probability)
output_dim = 1

x1 = Dense(output_dim)(x)
x1 = Activation('sigmoid', name='a1')(x1)

x2 = Dense(output_dim)(x)
x2 = Activation('sigmoid', name='a2')(x2)
  
x3 = Dense(output_dim)(x)
x3 = Activation('sigmoid', name='a3')(x3)
  
x4 = Dense(output_dim)(x)
x4 = Activation('sigmoid', name='a4')(x4)
  
x5 = Dense(output_dim)(x)
x5 = Activation('sigmoid', name='a5')(x5)
  
x6 = Dense(output_dim)(x)
x6= Activation('sigmoid', name='a6')(x6)
  
x7 = Dense(output_dim)(x)
x7 = Activation('sigmoid', name='a7')(x7)
  
x8 = Dense(output_dim)(x)
x8 = Activation('sigmoid', name='a8')(x8)
  
x9 = Dense(output_dim)(x)
x9 = Activation('sigmoid', name='a9')(x9)
  
x10 = Dense(output_dim)(x)
x10 = Activation('sigmoid', name='a10')(x10)
  
x11 = Dense(output_dim)(x)
x11 = Activation('sigmoid', name='a11')(x11)
  
x12 = Dense(output_dim)(x)
x12 = Activation('sigmoid', name='a12')(x12)
  
x13= Dense(output_dim)(x)
x13 = Activation('sigmoid', name='a13')(x13)
  
x14 = Dense(output_dim)(x)
x14 = Activation('sigmoid', name='a14')(x14)
  
x15 = Dense(output_dim)(x)
x15 = Activation('sigmoid', name='a15')(x15)
  
x16 = Dense(output_dim)(x)
x16 = Activation('sigmoid', name='a16')(x16)
  
x17 = Dense(output_dim)(x)
x17 = Activation('sigmoid', name='a17')(x17)
  
x18 = Dense(output_dim)(x)
x18 = Activation('sigmoid', name='a18')(x18)
  
x19 = Dense(output_dim)(x)
x19 = Activation('sigmoid', name='a19')(x19)
  
x20 = Dense(output_dim)(x)
x20 = Activation('sigmoid', name='a20')(x20)
  
x21 = Dense(output_dim)(x)
x21 = Activation('sigmoid', name='a21')(x21)

model = Model(inputs=[img_input], outputs=[x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21])
#print(model.summary())


# **4. Train model**

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, decay=1e-6)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])
                          
history = model.fit( my_training_batch_generator,
                    epochs = nb_epoch,
                    steps_per_epoch = steps_per_epoch,
                    verbose = True, 
                    validation_data = my_validation_batch_generator,
                    validation_steps = validation_steps)

Epoch 1/20
116/116 [==============================] - 55s 331ms/step - loss: 15.0605 - a1_loss: 0.5241 - a2_loss: 0.5351 - a3_loss: 0.6453 - a4_loss: 0.6095 - a5_loss: 0.4907 - a6_loss: 0.7205 - a7_loss: 1.1556 - a8_loss: 0.7766 - a9_loss: 0.6875 - a10_loss: 0.8167 - a11_loss: 0.6293 - a12_loss: 0.5546 - a13_loss: 1.0228 - a14_loss: 0.4971 - a15_loss: 1.0077 - a16_loss: 0.7996 - a17_loss: 0.9082 - a18_loss: 0.7509 - a19_loss: 0.4894 - a20_loss: 0.4663 - a21_loss: 0.9729 - a1_binary_accuracy: 0.7560 - a2_binary_accuracy: 0.7471 - a3_binary_accuracy: 0.6283 - a4_binary_accuracy: 0.6704 - a5_binary_accuracy: 0.7983 - a6_binary_accuracy: 0.5603 - a7_binary_accuracy: 0.2741 - a8_binary_accuracy: 0.5320 - a9_binary_accuracy: 0.5923 - a10_binary_accuracy: 0.4611 - a11_binary_accuracy: 0.6545 - a12_binary_accuracy: 0.7337 - a13_binary_accuracy: 0.3373 - a14_binary_accuracy: 0.7848 - a15_binary_accuracy: 0.3199 - a16_binary_accuracy: 0.5029 - a17_binary_accuracy: 0.4076 - a18_binary_accuracy: 0

In [ ]:
#Plot Loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='validation loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')


In [ ]:
#Plot results
plt.plot(history.history['binary_accuracy'], label='binary_accuracy')
plt.plot(history.history['val_binary_accuracy'], label = 'val_binary_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.legend(loc='lower right')

In [ ]:
# Test evaluation

#read gt_test
parking_test_import = pandas.read_csv( route + "/test/groundtruth.txt", delim_whitespace=True)
parking_test_data = np.array(parking_test_import.values)

parking_test_images = []
parking_test_labels = []
for i in range (len(parking_test_data)):
    image_name = parking_test_data[i][0]            
    image_name =  route + '/test/' + image_name        
    img = load_img(image_name, color_mode="rgb", target_size=(img_height, img_width), interpolation="nearest")     
    img = img_to_array(img)        
    parking_test_images.append(img)  
    parking_test_labels.append(parking_data[i][1:])

test_images = np.array(parking_test_images)
test_images = test_images / 255.0
parking_test_labels = np.array(parking_test_labels).astype('float32')



test_loss, test_acc = model.evaluate(test_images,  parking_test_labels, verbose=2)
print('Test accuracy: ',test_acc)